##랭체인을 이용한 텍스트 청킹
랭체인의 청킹 방식은 여러가지가 존재하지만 그 중 가장 대표적인 방식으로 크게 두 가지가 있습니다. 바로 길이 단위로 자르는 RecursiveCharacterTextSplitter 와 의미를 고려하여 자르는 SemanticChunker 입니다. 단순히 길이 단위로 자르기 때문에 정보 손실이 발생할 수 있는 RecursiveCharacterTextSplitter 와 의미 단위로 자르기는 하지만 성능이 매우 빈약한 SemanticChunker 를 사용하는 것보다는 더 나은 방법을 찾으셔야 합니다. 랭체인의 도구들은 일종의 베이스라인 도구로 삼으시기 바랍니다.

In [ ]:
!pip install langchain_openai langchain_experimental

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.7 MB/s eta 0:00:00


###1. 청킹
ChatGPT 와 같은 언어 모델들은 입력 길이에 제한이 있어 (요즘은 매우 넉넉하기는 하더라도), 정해진 입력 길이를 초과하는 문서는 한 번에 처리할 수 없습니다. 또한, 입력 길이 제한 내의 텍스트라도 너무 길면 모델이 검토해야 할 정보량이 과다해져 답변의 정확도가 떨어질 수 있습니다. 이러한 이유로 RAG 에서는 긴 입력 문서를 작은 단위로 분할하여 처리하게 됩니다. 이렇게 텍스트를 자르는 것을 일반적으로 청킹(Chunking) 이라고 부르며, 이 청킹을 위해서 다양한 전략들이 존재합니다.

###2.길이 단위 청킹 -RecursiveCharacterTextSplitter
랭체인에서 제공하는 청킹. 즉, 텍스트 분할 도구 중 가장 널리 사용되는 것은 RecursiveCharacter‐
TextSplitter 입니다. 이 도구는 긴 텍스트를 입력받아 특정 규칙에 따라 반복적으로 분할하여 더 짧은 단위의 텍스트 청크 (chunk) 로 만듭니다.
사용자는 각 청크의 최대 길이를 지정할 수 있습니다. 예를 들어, 길이가 10,000 인 텍스트에 대해 각 청크의 최대 길이를 500 으로 설정하면, 해당 텍스트는 길이 500 을 초과하지 않는 여러 개의 청크로 분할됩니다. 이 분할기는 ["\n\n", "\n", "", ""] 총 4 개의 문자를 기준으로 사용합니다. 동작 방식은 다음과 같습니다.

1. 먼저 \n\n(두 번의 줄바꿈) 을 기준으로 텍스트를 나눕니다.

2. 나눈 청크가 여전히 원하는 길이보다 크다면, \n(한 번의 줄바꿈)을 기준으로 다시 나눕니다.

3. 그래도 지정된 크기를 초과한다면, ” ”(공백) 을 사용하여 나누는 작업을 반복합니다.

RecursiveCharacterTextSplitter 는 이 과정을 통해 텍스트를 점진적으로 더 작은 단위로 분할하여, 최종
적으로 사용자가 원하는 크기에 근사한 청크들을 얻습니다. 이로써 긴 문서를 효과적으로 처리할 수 있는
작은 단위의 텍스트로 분할할 수 있습니다.

####1) 데이터 다운로드

굉장히 긴 길이를 가진 텍스트를 다운로드하고 길이를 출력합니다.

In [ ]:
import urllib.request
from langchain.text_splitter import RecursiveCharacterTextSplitter

urllib.request.urlretrieve("https://raw.githubusercontent.com/lovit/soynlp/master/tutorials/2016-10-20.txt",
                           filename="2016-10-20.txt")
with open("2016-10-20.txt", encoding='utf-8') as f:
    file = f.read()
print('텍스트의 길이: ',len(file))

텍스트의 길이:  18085369


텍스트의 길이가 18,085,369 로, 일반적으로 ChatGPT 같은 언어 모델에 한 번에 넣을 수 없는 굉장히 긴 길이의 텍스트입니다. 이제 이 텍스트를 ChatGPT 같은 언어 모델이 처리할 수 있는 적당한 길이로 분할해 봅시다.

####2) RecursiveCharacterTextSplitter
RecursiveCharacterTextSplitter() 를 이용하여 텍스트를 분할하는 text_splitter 객체를 만듭니다. 이때 chunk_size 의 값을 500 으로 지정하면 앞으로 text_splitter 로 텍스트를 분할할 때 각 분할된 청크는 길이가 500 을 결코 넘지 않습니다. chunk_overlap 은 텍스트를 분할할 때 각 청크가 내용을 얼만큼 겹치게 할 것인지를 정하는 값으로 0 을 지정하면 각 청크의 내용이 겹치지 않습니다.

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)

이렇게 선언된 text_splitter 로 파이썬 문자열을 분할할 때는 create_documents() 를 사용합니다.

In [ ]:
text = text_splitter.create_documents([file])
print('분할된 청크의 수: ',len(text))

분할된 청크의 수:  47068


47,068 개의 청크로 분할되었습니다. 1 번 인덱스의 청크를 출력하여 그 결과를 확인해봅시다.

In [ ]:
text[1]

Document(metadata={}, page_content='오패산터널 총격전 용의자 검거 서울 연합뉴스 경찰 관계자들이 19일 오후 서울 강북구 오패산 터널 인근에서 사제 총기를 발사해 경찰을 살해한 용의자 성모씨를 검거하고 있다 성씨는 검거 당시 서바이벌 게임에서 쓰는 방탄조끼에 헬멧까지 착용한 상태였다 독자제공 영상 캡처 연합뉴스  서울 연합뉴스 김은경 기자 사제 총기로 경찰을 살해한 범인 성모 46 씨는 주도면밀했다  경찰에 따르면 성씨는 19일 오후 강북경찰서 인근 부동산 업소 밖에서 부동산업자 이모 67 씨가 나오기를 기다렸다 이씨와는 평소에도 말다툼을 자주 한 것으로 알려졌다  이씨가 나와 걷기 시작하자 성씨는 따라가면서 미리 준비해온 사제 총기를 이씨에게 발사했다 총알이 빗나가면서 이씨는 도망갔다 그 빗나간 총알은 지나가던 행인 71 씨의 배를 스쳤다  성씨는 강북서 인근 치킨집까지 이씨 뒤를 쫓으며 실랑이하다 쓰러뜨린 후 총기와 함께 가져온 망치로 이씨 머리를 때렸다  이 과정에서 오후 6시 20분께 강북구 번동 길 위에서')

출력 결과를 보면 Document(page_content=“텍스트”) 의 형태를 가집니다. 이 형태는 랭체인을 이용하
여 텍스트를 다수의 청크로 분할했을 때 갖게 되는 형태로, 앞으로 랭체인 실습을 하면서 자주 보게 될 형
태이므로 기억해둡시다. 이때 원문에 접근하려면 각 청크에.page_content 를 붙여서 출력하면 됩니다.

In [ ]:
text[1].page_content

'오패산터널 총격전 용의자 검거 서울 연합뉴스 경찰 관계자들이 19일 오후 서울 강북구 오패산 터널 인근에서 사제 총기를 발사해 경찰을 살해한 용의자 성모씨를 검거하고 있다 성씨는 검거 당시 서바이벌 게임에서 쓰는 방탄조끼에 헬멧까지 착용한 상태였다 독자제공 영상 캡처 연합뉴스  서울 연합뉴스 김은경 기자 사제 총기로 경찰을 살해한 범인 성모 46 씨는 주도면밀했다  경찰에 따르면 성씨는 19일 오후 강북경찰서 인근 부동산 업소 밖에서 부동산업자 이모 67 씨가 나오기를 기다렸다 이씨와는 평소에도 말다툼을 자주 한 것으로 알려졌다  이씨가 나와 걷기 시작하자 성씨는 따라가면서 미리 준비해온 사제 총기를 이씨에게 발사했다 총알이 빗나가면서 이씨는 도망갔다 그 빗나간 총알은 지나가던 행인 71 씨의 배를 스쳤다  성씨는 강북서 인근 치킨집까지 이씨 뒤를 쫓으며 실랑이하다 쓰러뜨린 후 총기와 함께 가져온 망치로 이씨 머리를 때렸다  이 과정에서 오후 6시 20분께 강북구 번동 길 위에서'

2번 청크를 출력해봅시다.


In [ ]:
text[2].page_content

'사람들이 싸우고 있다 총소리가 났다 는 등의 신고가 여러건 들어왔다  5분 후에 성씨의 전자발찌가 훼손됐다는 신고가 보호관찰소 시스템을 통해 들어왔다 성범죄자로 전자발찌를 차고 있던 성씨는 부엌칼로 직접 자신의 발찌를 끊었다  용의자 소지 사제총기 2정 서울 연합뉴스 임헌정 기자 서울 시내에서 폭행 용의자가 현장 조사를 벌이던 경찰관에게 사제총기를 발사해 경찰관이 숨졌다 19일 오후 6시28분 강북구 번동에서 둔기로 맞았다 는 폭행 피해 신고가 접수돼 현장에서 조사하던 강북경찰서 번동파출소 소속 김모 54 경위가 폭행 용의자 성모 45 씨가 쏜 사제총기에 맞고 쓰러진 뒤 병원에 옮겨졌으나 숨졌다 사진은 용의자가 소지한 사제총기  신고를 받고 번동파출소에서 김창호 54 경위 등 경찰들이 오후 6시 29분께 현장으로 출동했다 성씨는 그사이 부동산 앞에 놓아뒀던 가방을 챙겨 오패산 쪽으로 도망간 후였다  김 경위는 오패산 터널 입구 오른쪽의 급경사에서 성씨에게 접근하다가 오후 6시'

chunk_overlap 의 값이 0 이었으므로 1 번 청크와 2 번 청크는 내용이 겹치지 않고, 1 번 청크 다음 내용이
2 번 청크로 이어집니다. 1 번 청크와 2 번 청크 모두 길이를 출력해보겠습니다.

In [ ]:
print('1번 청크의 길이: ',len(text[1].page_content))
print('2번 청크의 길이: ',len(text[2].page_content))

1번 청크의 길이:  498
2번 청크의 길이:  496


####3) chunk_overlap 사용하기
chunk_size 의 값이 500 이었으므로 두 개의 청크 모두 길이가 500 을 넘지 않습니다. 이번에는
chunk_overlap 을 이해하기 위해서 chunk_overlap 의 값을 50 으로 변경하여 text_splitter 를 다시
선언하고 create_documents() 를 사용하여 다수의 청크로 분할해보겠습니다.

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
text = text_splitter.create_documents([file])
print('분할된 청크의 수: ',len(text))

분할된 청크의 수:  48670


48,670 개의 청크로 분할되었습니다. 1 번 청크와 2 번 청크를 출력하여 그 결과를 확인해봅시다.

In [ ]:
text[1].page_content

'오패산터널 총격전 용의자 검거 서울 연합뉴스 경찰 관계자들이 19일 오후 서울 강북구 오패산 터널 인근에서 사제 총기를 발사해 경찰을 살해한 용의자 성모씨를 검거하고 있다 성씨는 검거 당시 서바이벌 게임에서 쓰는 방탄조끼에 헬멧까지 착용한 상태였다 독자제공 영상 캡처 연합뉴스  서울 연합뉴스 김은경 기자 사제 총기로 경찰을 살해한 범인 성모 46 씨는 주도면밀했다  경찰에 따르면 성씨는 19일 오후 강북경찰서 인근 부동산 업소 밖에서 부동산업자 이모 67 씨가 나오기를 기다렸다 이씨와는 평소에도 말다툼을 자주 한 것으로 알려졌다  이씨가 나와 걷기 시작하자 성씨는 따라가면서 미리 준비해온 사제 총기를 이씨에게 발사했다 총알이 빗나가면서 이씨는 도망갔다 그 빗나간 총알은 지나가던 행인 71 씨의 배를 스쳤다  성씨는 강북서 인근 치킨집까지 이씨 뒤를 쫓으며 실랑이하다 쓰러뜨린 후 총기와 함께 가져온 망치로 이씨 머리를 때렸다  이 과정에서 오후 6시 20분께 강북구 번동 길 위에서'

In [ ]:
text[2].page_content

'망치로 이씨 머리를 때렸다  이 과정에서 오후 6시 20분께 강북구 번동 길 위에서 사람들이 싸우고 있다 총소리가 났다 는 등의 신고가 여러건 들어왔다  5분 후에 성씨의 전자발찌가 훼손됐다는 신고가 보호관찰소 시스템을 통해 들어왔다 성범죄자로 전자발찌를 차고 있던 성씨는 부엌칼로 직접 자신의 발찌를 끊었다  용의자 소지 사제총기 2정 서울 연합뉴스 임헌정 기자 서울 시내에서 폭행 용의자가 현장 조사를 벌이던 경찰관에게 사제총기를 발사해 경찰관이 숨졌다 19일 오후 6시28분 강북구 번동에서 둔기로 맞았다 는 폭행 피해 신고가 접수돼 현장에서 조사하던 강북경찰서 번동파출소 소속 김모 54 경위가 폭행 용의자 성모 45 씨가 쏜 사제총기에 맞고 쓰러진 뒤 병원에 옮겨졌으나 숨졌다 사진은 용의자가 소지한 사제총기  신고를 받고 번동파출소에서 김창호 54 경위 등 경찰들이 오후 6시 29분께 현장으로 출동했다 성씨는 그사이 부동산 앞에 놓아뒀던 가방을 챙겨 오패산 쪽으로 도망간 후였다'

chunk_overlap 의 값을 50 으로 설정하면 기본적으로 각 청크는 앞, 뒤 청크 간 길이 약 50 정도의 내용이 겹치도록 구성됩니다. 실제로 청크 1 번의 마지막과 청크 2 번의 시작은’ 망치로 이씨 머리를 때렸다 이 과정에서 오후 6 시 20 분께 강북구 번동 길 위에서’ 라는 문장이 겹치도록 구성되어 있습니다.
지금까지 RecursiveCharacterTextSplitter() 를 통해 분할된 청크들을 보면 어떠한 문맥을 파악하여 문맥 단위로 분할하는 것이 아니라 기본적으로 길이에 맞추어 분할하므로 내용이 중간에 전개되다가 갑자기 끊긴다는 느낌을 받습니다. 이러한 분할 방식은 빠른 분할 결과를 얻을 수는 있지만 내용의 완결성이 없는 청크들로 인해 뒤에서 배우게 될 RAG(Retrieval‐Augmented Generation) 챗봇을 개발할 때 완결성 없는 청크들이 입력으로 전달되어 챗봇의 성능 저하 이슈를 발생시킵니다. 이러한 문제를 개선하기 위해서 많은 회사에서 텍스트를 짧은 청크로 분할할 때 의미를 파악하여 문맥 단위로 분할하려는 노력을 하고 있으며 랭체인에서도 이러한 기능을 제공하고 있습니다. 이제 RecursiveCharacterTextSplitter() 와는 달리 텍스트의 의미를 반영하여 분할하는 SemanticChunker()에 대해 알아봅시다.

###3. 의미 고려 청킹 -SemanticChunker
SemanticChunker 는 RecursiveCharacterTextSplitter 와 마찬가지로 긴 길이의 텍스트를 받아 더 짧은 단위 텍스트 청크로 분할하는 도구입니다. 다만, 앞서 설명했던 OpenAI 의 Embedding API 를 사용하여 각 문장을 임베딩 벡터로 변환하고 유사도를 구해서 유사한 문장끼리 그룹화하는 방식으로 동작합니다. 이렇게 분리된 청크들은 RecursiveCharacterTextSplitter 와는 달리 어느 정도 문맥의 의미가 고려되어 청크들이 분할된다는 특징이 있습니다.

####1) 데이터 다운로드
먼저 실습을 위해 필요한 랭체인 도구들을 임포트합니다. 내부적으로 OpenAI Embedding API 를 사
용하고 있으므로 OpenAIEmbeddings 와 SemanticChunker 를 임포트하고, 사용자의 OpenAI API 키 값
을 현재 실습 환경에 세팅합니다.

In [ ]:
import os
import urllib.request
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_experimental.text_splitter import SemanticChunker

os.environ['OPENAI_API_KEY'] ='Openai_api_key'

OpenAI 의 Embedding API 은 기본적으로 유료이므로 너무 긴 텍스트로 실습하지 않기를 권장합니다. 여기서는 이 책에서 제공하는 간단한 텍스트를 활용합니다. 이 책의 코드 저장소로부터 test.txt 파일을 다운로드하고 이를 읽어서 길이를 확인합니다.

In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/chatgpt-kr/openai-api-tutorial/main/ch06/test.txt",
                           filename="test.txt")

with open("test.txt", encoding="utf-8") as f:
  file = f.read()
print('텍스트의 길이:', len(file))

텍스트의 길이: 7460


####2) SemanticChunker사용하기
이제 SemanticChunker 를 이용하여 텍스트를 분할해봅시다. 객체를 만드는 것과 분할하는 방법은 RecursiveCharacterTextSplitter() 와 거의 동일합니다. SemanticChunker() 를 이용하여 텍스트를 분할하는 text_splitter 객체를 만듭니다. 이때 OpenAIEmbeddings() 를 전달하여 OpenAI 의 Embedding API를 사용한다는 것을 명시합니다.

In [ ]:
text_splitter = SemanticChunker(OpenAIEmbeddings())
texts = text_splitter.create_documents([file])
print('분할된 청크의 수: ', len(texts))

분할된 청크의 수:  10


총 10 개의 청크로 분할되었습니다. 임의로 3 번, 4 번, 5 번 청크를 출력하여 실제로 문맥이 바뀌는 구간을
포착하여 분할되는지 확인해보겠습니다.

In [ ]:
texts[3].page_content

'그의 끈기와 열정은 주위 사람들에게 큰 귀감이 되었다. 교수들은 그를 "우리 학과의 보물"이라고 불렀고, 후배들은 그에게 조언을 구하기 위해 줄을 섰다. 졸업할 때 그는 학과 수석의 영예를 안았다. 직업과 해외 진출 시도\n서울대를 졸업한 박민호는 큰 꿈을 안고 여러 IT 기업에 지원했다. 그는 자신의 능력을 인정받아 대기업에 입사할 수 있을 거라 믿었다.'

앞서 언급했듯이 랭체
인에서 문서를 청크로 분할했을 때 자주 보게 되는 형태이므로 기억해둡시다. 3 번 청크의 경우, 박민호라는 인물이 좋은 평판으로 졸업하여 IT 기업에 지원했다는 설명이 있습니다. 그리고 이어지는 4 번 청크는 다음과 같이 3 번 청크의 결말에서의 기대감과는 달리, 취업이 어려웠다는 내용을 담고 있습니다. 이는 4번 청크가 3 번 청크의 내용에서 반전이 되는 내용이므로 적절하게 의미가 달라질 때 잘라냈다고 볼 수도
있을 것입니다.

In [ ]:
texts[4].page_content

'그러나 현실은 냉혹했다. 연이은 탈락 통지에 그는 좌절감을 느꼈다. "내가 부족한 걸까?" 자신을 의심하기 시작했다. 결국 그는 작은 스타트업에서 일하기 시작했다. 급여는 많지 않았지만, 다양한 경험을 쌓을 수 있었다. 그는 이 시기를 자신의 실력을 갈고닦는 기회로 삼았다. 밤을 새워가며 새로운 기술을 익혔고, 회사 프로젝트에 혁신적인 아이디어를 제안했다. 2년이 지나자 그의 노력이 빛을 발하기 시작했다. 그가 개발한 알고리즘이 업계의 주목을 받게 된 것이다. 여러 기업에서 그에게 이직 제안을 해왔고, 그는 더 큰 회사로 옮길 수 있었다. 그러나 여전히 그의 꿈은 더 컸다. 그는 세계적인 IT 기업들이 모여 있는 미국 실리콘밸리로의 진출을 꿈꿨다. 수많은 지원서를 보냈지만, 대부분 거절당했다.'

그 후 5 번 청크의 경우 4 번 청크에서의 실패를 이겨내서 작은 회사에서부터 성장해가며 큰 회사로부터 스카우트 받기까지의 성장 과정을 그려내고 있습니다. 역시 4 번 청크에서의 내용에 또 다시 반전이 이루어지는 부분에서 적절하게 의미가 달라질 때 잘라냈습니다.

In [ ]:
texts[5].page_content

'언어 장벽과 경력 부족이 주된 이유였다. 그러나 박민호는 포기하지 않았다. 퇴근 후에는 영어 학원에 다녔고, 주말에는 국제 컨퍼런스에 참가해 네트워크를 넓혔다. 이러한 노력 끝에 2010년, 그는 마침내 미국 실리콘밸리의 한 중소 IT 기업에 취직하게 되었다. 실리콘밸리에서의 첫 직장은 그에게 새로운 도전과 기회를 제공했다. 그는 자신의 꿈을 이루기 위한 첫 발걸음을 내딛었다는 생각에 가슴이 뛰었다. 해외에서의 도전과 성공\n미국에서의 생활은 결코 쉬운 일이 아니었다. 문화적 차이와 언어 장벽, 그리고 새로운 환경에 적응해야 했다. 처음 몇 달 동안 박민호는 매일 밤 한국으로 돌아가고 싶다는 생각을 했다. 회의 중 동료들의 농담을 이해하지 못해 웃지 못할 때도 많았고, 업무 지시를 제대로 이해하지 못해 실수를 저지르기도 했다. 그러나 박민호는 포기하지 않았다. 그는 퇴근 후에도 영어 공부를 계속했고, 주말에는 현지 문화를 체험하기 위해 다양한 활동에 참여했다. 동료들과의 대화에서 놓친 부분이 있으면 나중에 따로 물어보며 이해하려 노력했다. 이러한 그의 성실함과 열정은 동료들에게 좋은 인상을 주었다. 점차 시간이 지나면서 박민호는 미국 생활에 적응해갔다. 그의 영어 실력도 크게 향상되었고, 회사에서의 업무 능력도 인정받기 시작했다. 특히 그는 인공지능 연구 분야에서 두각을 나타냈다. 그가 개발한 알고리즘은 회사의 주요 프로젝트에 적용되어 큰 성과를 거두었다. 이러한 성과를 바탕으로 박민호는 실리콘밸리의 여러 대기업에서 스카우트 제의를 받았다. 그는 끊임없이 자신의 한계를 시험하며 새로운 기술을 배우고 연구했다.'

SemanticChunker 가 문서를 분할하는 방법은 크게 세 가지가 있습니다.

• 백분위수 (Percentile) 방식 (기본값)

• 표준편차 (Standard Deviation) 방식

• 사분위수 (Interquartile) 방식

우선, 이 세 가지 방식은 모두 코사인 거리라는 개념을 사용합니다. 코사인 거리는 두 문장 간의 의미적 차이를 나타내는 척도입니다. 코사인 거리는 앞서 ‘코사인 유사도’ 에서 설명한 두 임베딩 벡터의 유사도인 코사인 유사도로부터 계산할 수 있습니다. 두 벡터의 코사인 유사도를 계산하고 1 에서 빼면 코사인 거리입니다.

• 코사인 거리 = 1 ‐ 코사인 유사도

세 가지 방식 중 하나를 적용하기 위해 먼저 각 문장은 임베딩 벡터로 변환되며, 인접한 문장 쌍 사이의 코사인 거리를 계산합니다. 이때 코사인 거리의 값의 범위는 0 에서 2 사이이며, 0 에 가까울수록 유사하고 2에 가까울수록 다릅니다.

####3) 백분위수 방식
백분위수 (Percentile) 방식은 SemanticChunker 가 따로 명시해주지 않으면 기본으로 사용하는 방식입니다. 따라서 앞의 실습에서는 백분위수 방식을 사용하여 총 10 개의 청크로 나누었던 것입니다.
백분위수 방식을 사용 할 때SemanticChunker() 를 이용한 text_splitter 선언 시 기준이 되는 백분위값에 해당하는 breakpoint_threshold_amount 의 값 을 임 의 로 설 정 할 수 있 습 니 다. break‐point_threshold_amount 의 기본값은 95 입니다. 따라서 다음 코드는 위의 실습에서 사용한 코드와 동일한 결과를 얻습니다.

In [ ]:
text_splitter = SemanticChunker(
    OpenAIEmbeddings(),
    breakpoint_threshold_type = "percentile",
    breakpoint_threshold_amount = 95,
)
texts = text_splitter.create_documents([file])
print('분할된 청크의 수: ', len(texts))

분할된 청크의 수:  10


앞에서 실습했던 것과 같이 청크의 수가 10 개가 나온 것을 확인할 수 있습니다. 백분위수 방식을 간단히 정리하면 다음과 같습니다.

- 기본값: breakpoint_threshold_amount = 95

- 원리:  모든 연속된 문장 쌍 사이의 코사인 거리를 계산합니다. 이 코사인 거리들을 크기 순으로 정렬합니다. 예를 들어 breakpoint_threshold_amount 의 값을 95 로 설
정한 경우, 95 번째 백분위수에 해당하는 코사인
거리를 찾습니다. 즉, 이렇게 찾은 값은 전체 거리
중 95% 가 이 값보다 작습니다. 이 값을 기준으
로 삼아 이보다 큰 코사인 거리를 가진 지점에서
텍스트를 나눕니다.

- 예시: 100 개의 문장이 있다면 99 개의 코사인 거리값이 생깁니다. 이 99 개의 값을 정렬하고 95 번째로 큰 값을 찾습니다. 이 값보다 큰 코사인 거리를 가진 곳에서만 텍스트를 나눕니다.

- 장점: 극단적으로 큰 의미적 차이가 있는 곳에서만 텍스트를 나누므로 주요 주제가 바뀌는 곳을 잘 찾을 수 있습니다.

####4) 표준편차 방식
이번에는 표준편차(Standard Deviation)방식을 사용해 봅시다. 표준 편차 방식을 사용하는 경우에는 breakpoint_threshold_type의 값으로 standard_deviation을 사용하면 됩니다. 이때 break_point_threshold_amount의 기본값은 3 입니다.

In [ ]:
text_splitter = SemanticChunker(
    OpenAIEmbeddings(),
    breakpoint_threshold_type = "standard_deviation",
    breakpoint_threshold_amount = 3,
)
texts = text_splitter.create_documents([file])
print('분할된 청크의 수: ', len(texts))


분할된 청크의 수:  3


백분위수 방식을 사용했을 때와 달리 청크의 수가 3 개로 줄었습니다. 표준편차 방식을 간단히 정리하면 다음과 같습니다.

- 기본값: breakpoint_threshold_amount = 3

- 원리: 모든 연속된 문장 쌍 사이의 코사인 거리를 계산합니다. 이 코사인 거리들의 평균과 표준편차를 계산합니다. 예를 들어
breakpoint_threshold_amount 의 값을 3 으로
설정한 경우 (평균 + 3 * 표준편차) 이 기준값입니
다. 이 기준값보다 큰 코사인 거리를 가진 지점
에서 텍스트를 나눕니다.

- 예시: 코사인 거리의 평균이 0.5 이고 표준편차가 0.1이라면, 기준값은 0.5 + (3 * 0.1) = 0.8 이 됩니다. 0.8 보다 큰 코사인 거리를 가진 곳에서 텍스트를 나눕니다.

- 장점: 텍스트 전체의 의미적 흐름을 고려하여 평균적인 문장 간 차이보다 훨씬 큰 변화가 있는 지점을 찾아냅니다. 이는 새로운 주제의 시작, 논점의 전환, 또는 이야기의 큰 전환점 등을 효과적으로 감지할 수 있게 해줍니다.

####5) 사분위수 방식
사분위수 (interquartile)방식을 사용해 봅시다. 사분위수 방식을 사용하는 경우에는 break_point_threshold_type의 값으로 interquartile을 사용하면 됩니다. 이때 breakpoint_threshold_amount의 기본값은 1.5입니다.

In [ ]:
text_splitter = SemanticChunker(
    OpenAIEmbeddings(),
    breakpoint_threshold_type = "interquartile",
    breakpoint_threshold_amount = 1.5
)
texts = text_splitter.create_documents([file])
print('분할된 청크의 수: ', len(texts))

분할된 청크의 수:  7


백분위수 방식을 사용했을 때와 달리 청크의 수가 7 개로 줄었습니다. 사분위수 방식을 간단히 정리하면 다음과 같습니다.

- 기본값: breakpoint_threshold_amount = 1.5

- 원리: 모든 연속된 문장 쌍 사이의 거리를 계산합니다. 이 거리들의 1 사분위수 (Q1, 25% 지점) 와 3 사분위수 (Q3, 75% 지점) 를 찾습니다. 사분위수 범위 (IQR) = Q3 ‐ Q1 를 계산합니다. 거리들의 평균에 (breakpoint_threshold_amount * IQR) 을 더한 값을 기준값으로 삼습니다. 이 기준값보다 큰 거리를 가진 지점에서 텍스트를 나눕니다.

- 예시: Q1 이 0.3, Q3 가 0.7, 평균이 0.5,
breakpoint_threshold_amount 가 1.5 라면
IQR = 0.7 ‐ 0.3 = 0.4 입니다. 기준값은 0.5 + (1.5 * 0.4) = 1.1 이 됩니다. 1.1 보다 큰 거리를 가진 곳에서 텍스트를 나눕니다.

- 장점: 텍스트의 전반적인 구조를 고려하면서도 지나치게 민감하지 않게 분할점을 찾습니다. 특히 긴 문서에서 주요 섹션의 경계를 식별하는 데 효과적입니다. 또한 이상치 (극단적으로 다른 부분) 에 덜민감하여 텍스트의 전반적인 흐름을 해치지 않으면서도 중요한 주제 전환을 포착할 수 있습니다.